In [1]:
import pandas as pd
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [2]:
jobs_df = pd.read_csv("/Users/jacobemerson/code/cgallin/cover_genie/raw_data/job_data/postings.csv")
jobs_df.head(1)

,job_id,company_name,title,description,max_salary,pay_period,location,company_id,views,med_salary,...,skills_desc,listed_time,posting_domain,sponsored,work_type,currency,compensation_type,normalized_salary,zip_code,fips
0,921716,Corcoran Sawyer Smith,Marketing Coordinator,Job descriptionA leading real estate firm in N...,20.0,HOURLY,"Princeton, NJ",2774458.0,20.0,NaN,...,Requirements: \n\nWe are seeking a College or ...,1.713398e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,38480.0,8540.0,34021.0


In [3]:
def preprocessor(df):
    def text_cleaner(df):
        # Ensure 'description' column exists
        if 'description' not in df.columns:
            raise ValueError("DataFrame must contain a 'description' column.")
        
        # Drop rows with missing 'description' values
        df_cleaned = df.dropna(subset=['description'])
        
        # Convert text to lowercase and remove punctuation
        translator = str.maketrans('', '', string.punctuation)
        df_cleaned['description'] = df_cleaned['description'].str.lower().str.translate(translator)
        
        # Remove numbers
        df_cleaned['description'] = df_cleaned['description'].str.replace(r'\d+', '', regex=True)
        
        return df_cleaned['description']

    text = text_cleaner(df)

    # Tokenize the text
    def tokenizer(texts):
        return texts.apply(word_tokenize)

    tokens = tokenizer(text)

    # Remove stopwords
    def remove_stopwords(tokens):
        stop_words = set(stopwords.words('english')).union(stopwords.words('french'))
        return tokens.apply(lambda words: [w for w in words if w not in stop_words])

    no_stopword_tokens = remove_stopwords(tokens)

    # Lemmatize text
    def lemmatize(tokens):
        lemmatizer = WordNetLemmatizer()
        
        def lemmatize_words(words):
            # Lemmatize verbs and nouns
            lemmatized_verbs = [lemmatizer.lemmatize(word, pos='v') for word in words]
            lemmatized_nouns = [lemmatizer.lemmatize(word, pos='n') for word in lemmatized_verbs]
            return lemmatized_nouns
        
        return tokens.apply(lemmatize_words)

    lemmatized_tokens = lemmatize(no_stopword_tokens)

    # Return lemmatized tokens as a new DataFrame column
    df['processed_description'] = lemmatized_tokens
    return pd.DataFrame(df['processed_description'])
    


In [4]:
job_desc_words = preprocessor(jobs_df[0:100])

/var/folders/34/6f_0fz3s055bpk99ysxrnnth0000gn/T/ipykernel_4923/995941137.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['processed_description'] = lemmatized_tokens


In [5]:
job_desc_words

,processed_description
0,"[job, descriptiona, lead, real, estate, firm, ..."
1,"[aspen, therapy, wellness, commit, serve, clie..."
2,"[national, exemplar, accept, application, assi..."
3,"[senior, associate, attorney, elder, law, trus..."
4,"[look, hvac, service, tech, experience, commer..."
...,...
95,"[qualificationsexperiencedata, entry, year, re..."
96,"[firmdynamic, srg, boutique, political, consul..."
97,"[global, sculpt, streetwear, fashion, brand, n..."
98,"[executive, clean, service, inc, maid, service..."


In [6]:
resumes_df = pd.read_csv('/Users/jacobemerson/code/cgallin/cover_genie/raw_data/archive-3/Resume/Resume.csv')

In [7]:
resumes_df = resumes_df.rename(columns={'Resume_str':'description'})

In [8]:
jobs_df['description'][0]

'Job descriptionA leading real estate firm in New Jersey is seeking an administrative Marketing Coordinator with some experience in graphic design. You will be working closely with our fun, kind, ambitious members of the sales team and our dynamic executive team on a daily basis. This is an opportunity to be part of a fast-growing, highly respected real estate brokerage with a reputation for exceptional marketing and extraordinary culture of cooperation and inclusion.Who you are:You must be a well-organized, creative, proactive, positive, and most importantly, kind-hearted person. Please, be responsible, respectful, and cool-under-pressure. Please, be proficient in Adobe Creative Cloud (Indesign, Illustrator, Photoshop) and Microsoft Office Suite. Above all, have fantastic taste and be a good-hearted, fun-loving person who loves working with people and is eager to learn.Role:Our office is a fast-paced environment. You’ll work directly with a Marketing team and communicate daily with ot

In [11]:
resume_words = preprocessor(resumes_df[0:100])

/var/folders/34/6f_0fz3s055bpk99ysxrnnth0000gn/T/ipykernel_4923/995941137.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['processed_description'] = lemmatized_tokens


In [13]:
resume_words

,processed_description
0,"[hr, administratormarketing, associate, hr, ad..."
1,"[hr, specialist, u, hr, operation, summary, ve..."
2,"[hr, director, summary, year, experience, recr..."
3,"[hr, specialist, summary, dedicate, drive, dyn..."
4,"[hr, manager, skill, highlight, hr, skill, hr,..."
...,...
95,"[director, hr, executive, profile, ambitious, ..."
96,"[hr, senior, specialist, career, overview, ded..."
97,"[hr, customer, service, representative, summar..."
98,"[hr, service, representative, summary, multisk..."


In [ ]:
print("hello")

hi
